In [ ]:
from google.colab import drive
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1nB2zTXfdYWs6f2jj0Iv_hZVIulWOky2K"})
downloaded.GetContentFile('diverse_sampling_2000.zip') 
!unzip '/content/diverse_sampling_2000.zip' -d '/content/diverse_sampling_2000' 

Archive:  /content/diverse_sampling_2000.zip
replace /content/diverse_sampling_2000/00188_Animalia_Arthropoda_Insecta_Coleoptera_Cantharidae_Cantharis_fusca/00dbea4b-77bb-4aad-8fb5-cdd0484d624d.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import glob
folder1 = "/content/diverse_sampling_2000/00188_Animalia_Arthropoda_Insecta_Coleoptera_Cantharidae_Cantharis_fusca/"
folder1_list = glob.glob(folder1+"*") # 200 sample Cantharis_fusca from slide 2
folder2 = "/content/diverse_sampling_2000/00333_Animalia_Arthropoda_Insecta_Coleoptera_Coccinellidae_Psyllobora_vigintimaculata/" 
folder2_list = glob.glob(folder2+"*") # 200 sample Psyllobora_vigintimaculata from slide 2 
folder3 = "/content/diverse_sampling_2000/00433_Animalia_Arthropoda_Insecta_Diptera_Bombyliidae_Poecilanthrax_lucifer/"
folder3_list = glob.glob(folder3+"*") # 200 samples of Poecilanthrax_lucifer from slide 2
folder4 = "/content/diverse_sampling_2000/00472_Animalia_Arthropoda_Insecta_Diptera_Syrphidae_Eristalis_transversa/"
folder4_list = glob.glob(folder4+"*") # 200 Eristalis_transversa from slide 2
folder5 = "/content/diverse_sampling_2000/00493_Animalia_Arthropoda_Insecta_Diptera_Syrphidae_Toxomerus_politus/"
folder5_list = glob.glob(folder5+"*") # 200 samples of Toxomerus_politus from slide 2
folder6 = "/content/diverse_sampling_2000/00577_Animalia_Arthropoda_Insecta_Hemiptera_Flatidae_Ormenoides_venusta/"
folder6_list = glob.glob(folder6+"*") # 200 sample of Ormenoides_venusta from slide 2
folder7 = "/content/diverse_sampling_2000/00702_Animalia_Arthropoda_Insecta_Hymenoptera_Apidae_Bombus_terrestris/"
folder7_list = glob.glob(folder7+"*") # 200 samples of Bombus_terrestris from slide 2
folder8 = "/content/diverse_sampling_2000/00823_Animalia_Arthropoda_Insecta_Hymenoptera_Vespidae_Polistes_apachus/"
folder8_list = glob.glob(folder8+"*") # 200 samples of Polistes_apachu from slide 2
folder9 = "/content/diverse_sampling_2000/01253_Animalia_Arthropoda_Insecta_Lepidoptera_Geometridae_Lomographa_vestaliata/"
folder9_list = glob.glob(folder9+"*") # 200 samples of Lomographa_vestaliata from slide 2
folder10 = "/content/diverse_sampling_2000/01797_Animalia_Arthropoda_Insecta_Lepidoptera_Nymphalidae_Boloria_dia/"
folder10_list = glob.glob(folder10+"*") # 200 samples of Boloria_dia from slide 2

# testing = "/content/testing/"
# testing_list = glob.glob(testing+'*')


In [ ]:
pip install open_clip_torch #installing open clip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from PIL import Image
import open_clip
from matplotlib.patches import bbox_artist
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='laion400m_e32')


100%|█████████████████████████████████████| 1.71G/1.71G [08:07<00:00, 3.51MiB/s]


In [ ]:

temp = [folder1_list,folder2_list,folder3_list,folder6_list,folder9_list,folder10_list] # Selection of 6 distinguishable classes [Cantharis_fusca,Psyllobora_vigintimaculata,Poecilanthrax_lucifer,Ormenoides_venusta, Lomographa_vestaliata, Boloria_dia]
curr = 0
lst = [1,2,3,6,9,10] # include the file number similar to the above list.
res =  []
# templates = ['itap of a',] # include different prefixes stated in clip paper 
# 'a bad photo of the',
# 'a origami',
# 'a photo of the large',
# 'art of the', 
# 'a photo of the small',]

accuracy = 0
# for curr in templates:
for number,filepath in enumerate(temp):
  for i,image in enumerate(filepath):
      filename = image.split('/')[-1]
      image = preprocess(Image.open(image)).unsqueeze(0)
      # print(curr+' black color beetle with orange head, a type of insect') # include this the input in this format if you want to use any templete
      text = open_clip.tokenize(['a photo of black color beetle with orange head, a type of insect','a photo of white bug with black color spots, a type of insect','a photo of yellow insect with brown wings , a type of insect','a photo of Light green insect, looks like petal','a photo of white color insect, a type of moth','a photo of  insect with black spots on brown wings, a type of butterfly']) #selected the template with highest accuracy 
      with torch.no_grad():
          image_features = model.encode_image(image)
          text_features = model.encode_text(text)
          text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1) 
      max = 0
      text_probslst = text_probs[0].tolist()
      print(text_probslst)
      for index in range(len(text_probslst)):
        if text_probslst[index]>max:
          max = text_probslst[index]
          final  = index
      if number == final:
        accuracy +=1
      print(text_probslst,final,res)
      print(number)
      print(accuracy)
      res.append(final+1)
print((accuracy//1200)*100)
ytrue = [1]*200 +[2]*200+[3]*200+[4]*200+[5]*200+[6]*200
print(len(ytrue))
print(len(res))
labels = ['folder1','folder2','folder3','folder4','folder5','folder6']
cm = confusion_matrix(ytrue, res)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(xticks_rotation=45)
plt.savefig('insect_classifier.png',bbox_inches = 'tight')
  

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 []
0
1
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1]
0
2
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1]
0
3
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1]
0
4
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1, 1]
0
5
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1, 1, 1]
0
6
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1, 1, 1, 1]
0
7
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1, 1, 1, 1, 1]
0
8
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1, 1, 1, 1, 1, 1]
0
9
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1, 1, 1, 1, 1, 1, 1]
0
10
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0
11
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0 [1, 1, 1

KeyboardInterrupt: ignored

In [ ]:
from matplotlib.patches import bbox_artist
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
ytrue = [1]*200 +[2]*200+[3]*200+[4]*200+[5]*200+[6]*200
print(len(ytrue))
print(len(res))
labels = ['folder1','folder2','folder3','folder4','folder5','folder6']
cm = confusion_matrix(ytrue, res)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(xticks_rotation=45)
plt.savefig('insect_classifier.png',bbox_inches = 'tight')

1200
103


ValueError: ignored